In [1]:
import numpy as np
import time

import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torch.autograd as autograd
import numpy as np

syft = sy 

hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
james = sy.VirtualWorker(hook, id="james")
crypto_provider = james
torch = th

# th.manual_seed(1)


_get_hooked_func torch manual_seed torch.manual_seed


In [2]:
def plan(func):
    def hooked_func(*args, **kwargs):
    
        syft.hook.trace, syft.hook.trace_inactive = True, True

        result = func(*args, **kwargs)

        syft.hook.trace, syft.hook.trace_inactive = False, False

        
        placeholders = {}
        var_count = [0]
        input_placeholders = []
        def add_placeholder(t, find_inputs=False):
            if t.id not in placeholders.keys():
                placeholder = sy.PlaceHolder(tags={f'#{var_count[0]+1}'})
                placeholders[t.id] = placeholder
                if find_inputs:
                    input_placeholders.append(placeholder)
                    placeholder.tags.add('#input')
                var_count[0] += 1
                    
            return placeholders[t.id]

        def replace_with_placeholders(obj, **kw):
            if isinstance(obj, (tuple, list)):
                r = [replace_with_placeholders(o, **kw) for o in obj]
                return type(obj)(r)
            elif isinstance(obj, dict):
                return {
                    key: replace_with_placeholders(value, **kw)
                    for key, value in obj.items()
                }
            elif isinstance(obj, th.Tensor):
                return add_placeholder(obj, **kw)
            elif isinstance(obj, (int, float, str, bool)):
                return obj
            elif obj is None:
                return None
            else:
                raise TypeError(f"Type {type(obj)} not supported in plans args/kwargs")


        new_logs = []
        for log in syft.trace_logs:
            req, resp = log
            req, resp = replace_with_placeholders(req, find_inputs=True), replace_with_placeholders(resp)
            new_logs.append((req, resp))
            
        print('==INPUTS==')
        print(input_placeholders)
        for log in new_logs:
            req, resp = log
            print('==========')
            print(req)
            print('-->', resp)
            
            
        print('\nInstanciating inputs to -0.7...\n')
        # Simulate instanciation
        for placeholder in input_placeholders:
            placeholder.instanciate(th.Tensor([-.7]))
            
        print('Running operations...\n')
        for i, log in enumerate(new_logs):
            print('run cmd', i)
            (cmd, self, args, kwargs), resp = log
            if self is None:
                r = eval(cmd)(*args, **kwargs)
            else:
                r = getattr(self, cmd)(*args, **kwargs)
            resp.instanciate(r.child)
            print(resp)
            

        syft.trace_logs = []
    
        return result
    
    return hooked_func

In [3]:
@plan
def foo(x, y):
    z = torch.add(x, y)
    r = z.abs()
    return r

In [4]:
x = th.Tensor([1.])
y = th.Tensor([2.])
m = foo(x, y)

==INPUTS==
[PlaceHolder(#1, #input), PlaceHolder(#2, #input)]
('torch.add', None, (PlaceHolder(#1, #input), PlaceHolder(#2, #input)), {})
--> PlaceHolder(#3)
('abs', PlaceHolder(#3), (), {})
--> PlaceHolder(#4)

Instanciating inputs to -0.7...

Running operations...

run cmd 0
PlaceHolder(#3)>tensor([-1.4000])
run cmd 1
PlaceHolder(#4)>tensor([1.4000])
